In [1]:
sc.stop()

# Start

In [2]:
import os

from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.types import Row
import pyspark

In [3]:
ss = SparkSession.builder\
                 .appName("BOSCH") \
                 .config("spark.sql.parquet.compression.codec","snappy")\
                 .config("spark.submit.deployMode","client")\
                 .config("spark.sql.shuffle.partitions","1000")\
                 .config("spark.executor.memory","8g")\
                 .config("spark.executor.memoryOverhead","2g")\
                 .enableHiveSupport()\
                 .getOrCreate()

In [4]:
sqlContext = SQLContext(ss)

## Directories

In [5]:
def get_paths(path):
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
    list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(path))
    result = [file.getPath().getName() for file in list_status]
    return result

In [6]:
class last_time_path():
    
    def __init__(self, path, time):
        self.path = path
        self.time = time

    def get_last_time(self):
        
        while True:
            
            self.path = self.path + "/" + get_paths(self.path)[-1]
            
            if self.time in get_paths(self.path)[-1]:
                self.path = self.path + "/" + get_paths(self.path)[-1]
                break
        
        last_path = self.path

        return last_path

In [7]:
dataDirectory = "/products/cri2/data/manufacturing/topics/cri2-mfg-streaming-results-v6"

In [8]:
last_path = last_time_path(dataDirectory, "hour")

In [9]:
input_dir = last_path.get_last_time()

In [16]:
print(input_dir)

/products/cri2/data/manufacturing/topics/cri2-mfg-streaming-results-v6/year=2019/month=05/day=15/hour=12


## Import Data

___

In [33]:
def import_data(inputpath):

    print("Done!, Value Count: " + str (data.count()))
    
    return sqlContext.read.parquet(inputpath)


In [34]:
data  = import_data(input_dir)

Done!, Value Count: 901033


# Peeking Data

---

In [ ]:
print(data.columns)

In [ ]:
selected_columns_list = ['LINE',
                    'LOCATION_AREA',
                    'LOCATION_FUNCTION',
                    'LOCATION_ID',
                    'LOCATION_LINE',
                    'LOCATION_PLANT',
                    'LOCATION_PRODUCT',
                    'LOCATION_RESULT_STATE',
                    'LOCATION_STATINDEX',
                    'LOCATION_STATION',
                    'LOCATION_TOOLPOSITION',
                    'LOCATION_WORKPOSITION',
                    'NIOBITS',
                    'PLANT',
                    'PRODUCT',
                    'PRODUCTION_DAY',
                    'REKURSIONFLAG',
                    'RESULT_DATE_LOCAL',
                    'RESULT_VALUE',
                    'TIME_STAMP',
                    'TYPE_NUMBER',
                    'PARAM_NAME']

In [ ]:
data_selected = data.select(selected_columns_list)
data_selected.count()

In [ ]:
data_filtered = data_selected.filter((data_selected["LOCATION_PLANT"] == "Bu2") & \
                                  (data_selected["PRODUCT"] == "Testing")& \
                                  (data_selected["LOCATION_STATION"] == "2")& \
                                  (data_selected["PARAM_NAME"] == "fcode"))

In [ ]:
data_filtered.count()